In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI
import panel as pn

load_dotenv()  # Load environment variables from a .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise EnvironmentError("OPENAI_API_KEY environment variable is not set.")

client = OpenAI(
  api_key=OPENAI_API_KEY
)
def completion_message(prompt):
    messages = [{'role':'user', 'content':prompt}]
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
    )
    return completion.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-4o-mini", temperature=0):
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return completion.choices[0].message.content

In [2]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user','content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant','content':f"{response}"})
    panels.append(
        pn.Row('User:',pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#000000'})))
    return pn.Column(*panels)


In [3]:
pn.extension()
panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
Paneer Tikka Pizza ₹350, ₹250, ₹175 \
Chicken Tikka Pizza  ₹400, ₹250, ₹180 \
Veg Kebab Pizza 350, ₹200, ₹150 \
Butter Chicken Pizza 450, ₹250, ₹175 \
Achari Paneer Pizza 350, ₹200, ₹150 \
Tandoori Chicken Pizza ₹400, ₹250, ₹180 \
Pav Bhaji Pizza ₹300, ₹150, ₹75 \
Desi Margherita with Indian Herbs ₹250, ₹150, ₹75 \
Malai Paneer Pizza ₹250, ₹150, ₹75 \
Spicy Schezwan Veg/Chicken Pizza ₹400, ₹250, ₹180 \
Drinks: \
bottled water ₹20 \
"""} ]


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=500),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=500, loading_indicator=True)